In [1]:
'''
    this file are mainly used the function from sklearn
    including classifier, feature selection, and plot
'''

import couchdb

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.feature_selection import chi2, SelectKBest, f_regression, f_classif
from collections import Counter
import operator
import json
import string
import os
import sys
import csv
import numpy as np
import datetime
import pickle
from couchdb.mapping import Document, TextField, IntegerField, DateTimeField, BooleanField

from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import configparser



In [4]:
'''
    connect to db and read data from db
'''

remote_ip = 'http://115.146.92.169:5984'
couch_azure = couchdb.Server(remote_ip)
remote_train_db = couch_azure['train_data']
print(remote_train_db)

texts = []
labels = []
for _id in remote_train_db:
    tweet = remote_train_db.get(_id)
    texts.append(tweet['text'])
    labels.append(tweet['label'])

In [5]:
'''
    store counter and tf-id file
'''
TFID_FILE = 'tfid_counter_model40000_f_class_cloud.pkl'
WRITE = 'wb'
COUNTER_FILE = 'counter_model40000_f_class_cloud.pkl'
WORD_LEVEL = 'word'

X_train_features = []
X_train_features_tfid = []
down_texts = []
down_labels = []


# get best 5000 vocabulary from 7000
vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5)
X_train_features = vectorizer.fit_transform(texts)
ch2 = SelectKBest(f_classif, k = 6000)
X_train_features = ch2.fit_transform(X_train_features, labels)
voc = np.asarray(vectorizer.get_feature_names())[ch2.get_support()]

# save the model
vectorizer = TfidfVectorizer(sublinear_tf = True, max_df = 0.5, vocabulary = voc)
X_train_features_tfid = vectorizer.fit_transform(texts)
pickle.dump(vectorizer.vocabulary_,open(TFID_FILE, WRITE))
vectorizer = CountVectorizer(analyzer = WORD_LEVEL, vocabulary = voc)
X_train_features = vectorizer.fit_transform(texts)
pickle.dump(vectorizer.vocabulary_,open(COUNTER_FILE, WRITE))

X_train_features = X_train_features.toarray()

X_train_features_tfid = X_train_features_tfid.toarray()

In [6]:
'''
    balance the file
'''
count_pos = 0
count_neg = 0
count_neu = 0
down_texts = []
down_labels = []
for i in range(len(labels)):
    if labels[i] == 1 and count_pos < 50000:
        down_texts.append(X_train_features[i])
        down_labels.append(labels[i])
        count_pos += 1
    elif labels[i] == -1:
        down_texts.append(X_train_features[i])
        down_labels.append(labels[i])
        count_neg += 1
    elif labels[i] == 0 and count_neu < 50000:
        down_texts.append(X_train_features[i])
        down_labels.append(labels[i])
        count_neu += 1
print(count_pos)
print(count_neg)
print(count_neu)

In [10]:
'''
    logistic regression
    show cross validation first
    then hold-out
'''
lr = LogisticRegression()
scores = cross_val_score(lr, down_texts, down_labels, cv=10)
score = 0.0
for item in scores:
	score += item
print(score/10)

X_train, X_test, y_train, y_test = train_test_split(
	down_texts, down_labels, test_size=0.2, random_state=42)

# logistic regression for cross validation


lr.fit(X_train, y_train)
result = lr.predict(X_test)
lr.score(X_test, y_test)

In [11]:
'''
    save logistic regression
'''
counter_file = 'counter_f_classi_40000_lr_cloud.pkl'
write_byte = 'wb'
with open(counter_file, write_byte) as fid:
    pickle.dump(lr, fid)

In [12]:
'''
    multinomial naive bayes
    show cross validation first
    then hold-out
'''
clf = MultinomialNB()
scores = cross_val_score(clf, down_texts, down_labels, cv=10)
score = 0.0
for item in scores:
	score += item
print(score/10)

clf.fit(X_train, y_train)
clf.score(X_test, y_test)

In [13]:
'''
    save multinomial naive bayes
'''
counter_file = 'counter_f_class_6000_40000neg_mnb_cloud.pkl'
write_byte = 'wb'
with open('counter_f_class_6000_40000neg_mnb_cloud.pkl', 'wb') as fid:
    pickle.dump(clf, fid)

In [14]:
'''
    neural network
    show hold-out
'''
from sklearn.neural_network import MLPClassifier
neural = MLPClassifier(solver='sgd', alpha=1e-5, activation = 'tanh',
                     hidden_layer_sizes=(3, 3), random_state=1, verbose = True)

neural.fit(X_train, y_train) 
neural.score(X_test, y_test)

In [15]:
'''
    show learning curve with logistic regression
'''

import numpy as np
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit


def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):

    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt


title = "Learning Curves (Logistic regression chi2)"

cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = logistic regression()
plot_learning_curve(estimator, title, X_train, y_train, ylim=(0.7, 1.01), cv=10)



plt.show()

In [16]:
'''
    show learning curve with Multinomial Naive Bayes
'''

title = "Learning Curves (Multinomial Naive Bayes chi2)"

cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

estimator = MultinomialNB()
plot_learning_curve(estimator, title, X_train, y_train, ylim=(0.7, 1.01), cv=10)

plt.show()

In [ ]:
'''
    show learning curve with Neural Network
'''

title = "Learning Curves (Neural Network chi 2)"

cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)

new_estimator = MLPClassifier(solver='sgd', alpha=1e-5, activation = 'tanh',
                     hidden_layer_sizes=(3, 3), random_state=1, verbose = True, shuffle=True, early_stopping=True)

plot_learning_curve(new_estimator, title, down_texts, down_labels, ylim=(0.7, 1.01), cv=10)



plt.show()